In [268]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style
import math
import random

In [269]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model", LpMaximize)

In [270]:
#PARAMETERS

T=12 # end of promotion horizon
PHorizon = list(range(0, T))

I=10 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

Demand = np.zeros((I, T), dtype = int)

for i in Products:
    for t in PHorizon:
        Demand[i][t] = random.randint(0, 10000)
    
Promo_Sensitivity = np.zeros(I, dtype = float)
for i in Products:
    Promo_Sensitivity[i] = random.uniform(0.75, 1)


Total_PaidQty_Limit = 1000000
Total_FreeQty_Limit = 250000

Promo_Sensitivity

array([0.97671939, 0.76227342, 0.81819194, 0.83515284, 0.8657577 ,
       0.89283139, 0.95198538, 0.80268148, 0.90668868, 0.87407859])

In [271]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 20000
P = LpVariable.dicts("Paid_Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 5000
F = LpVariable.dicts("Free_Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

#Promo Ratio
#R = LpVariable.dicts("Promo_Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')

In [272]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon)

Promo_Model += Total_Paid_Quantity

In [273]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
        Promo_Model += P[i][t] + F[i][t] <= Demand[i][t]*(1 + Promo_Sensitivity[i]*F[i][t]*(1/FreeQtyLimit))
    
#NO FREE QTY WITHOUT PROMOTION
M = FreeQtyLimit
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] <= X[i][t]*M
        Promo_Model += F[i][t] >= X[i][t]-0.5

#PROMOTED PRODUCTS AT TIME "t"
Promoted_Products = 7
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) <= Promoted_Products
    
#PROMO PERIOD FOR PRODUCT "i"  
Max_Promo_Duration = 4
Min_Promo_Duration = 1
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) <= Max_Promo_Duration
    Promo_Model += lpSum(X[i][t] for t in PHorizon) >= Min_Promo_Duration
    
#TOTAL PAID QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(P[i][t] for t in PHorizon) <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(F[i][t] for t in PHorizon) <= Total_FreeQty_Limit

In [274]:
solver = GUROBI()
solver.solve(Promo_Model)

# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 632 rows, 360 columns and 3960 nonzeros
Model fingerprint: 0x375c6803
Variable types: 0 continuous, 360 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-01, 1e+06]
Found heuristic solution: objective 639628.00000
Presolve removed 600 rows and 240 columns
Presolve time: 0.00s
Presolved: 32 rows, 120 columns, 360 nonzeros
Found heuristic solution: objective 724602.00000
Variable types: 0 continuous, 120 integer (120 binary)
Found heuristic solution: objective 728719.00000

Root relaxation: cutoff, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [275]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  728719.0 

Free_Quantity_0_0 = 0.0
Free_Quantity_0_1 = 5000.0
Free_Quantity_0_10 = 5000.0
Free_Quantity_0_11 = 0.0
Free_Quantity_0_2 = 5000.0
Free_Quantity_0_3 = 5000.0
Free_Quantity_0_4 = 0.0
Free_Quantity_0_5 = 0.0
Free_Quantity_0_6 = 0.0
Free_Quantity_0_7 = 0.0
Free_Quantity_0_8 = 0.0
Free_Quantity_0_9 = 0.0
Free_Quantity_1_0 = 0.0
Free_Quantity_1_1 = 0.0
Free_Quantity_1_10 = 0.0
Free_Quantity_1_11 = 0.0
Free_Quantity_1_2 = 5000.0
Free_Quantity_1_3 = 5000.0
Free_Quantity_1_4 = 0.0
Free_Quantity_1_5 = 0.0
Free_Quantity_1_6 = 5000.0
Free_Quantity_1_7 = 0.0
Free_Quantity_1_8 = 5000.0
Free_Quantity_1_9 = 0.0
Free_Quantity_2_0 = 5000.0
Free_Quantity_2_1 = 0.0
Free_Quantity_2_10 = 0.0
Free_Quantity_2_11 = 0.0
Free_Quantity_2_2 = 5000.0
Free_Quantity_2_3 = 0.0
Free_Quantity_2_4 = 0.0
Free_Quantity_2_5 = 5000.0
Free_Quantity_2_6 = 0.0
Free_Quantity_2_7 = 5000.0
Free_Quantity_2_8 = 0.0
Free_Quantity_2_9 = 0.0
Free_Quantity_3_0 = 0.0
Free_Quantity_3_1 = 0.0
Free

In [276]:
#PROMO RATIO IS CALCULATED BASED ON THE OPTIMUM P[i][t] AND F[i][t] VALUES
Promo_Ratio = np.zeros((I, T), dtype = float)

for i in Products:
    for t in PHorizon:
        Promo_Ratio[i][t] = (F[i][t].varValue)/(P[i][t].varValue + F[i][t].varValue)

df_Promo_Ratio = pd.DataFrame(Promo_Ratio,
                 index=range(1,I+1),
                 columns=range(1,T+1))
df_Promo_Ratio

,1,2,3,4,5,6,7,8,9,10,11,12
1,0.000000,0.267924,0.292774,0.289788,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.311701,0.000000
2,0.000000,0.000000,0.405154,0.381301,0.000000,0.000000,0.294325,0.000000,0.327418,0.000000,0.000000,0.000000
3,0.338730,0.000000,0.335503,0.000000,0.000000,0.369686,0.000000,0.335211,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.334560,0.000000,0.453803,0.000000,0.000000,0.000000,0.000000,0.376449
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.289570,0.300445,0.000000,0.296384,0.290596,0.000000
6,0.292090,0.281120,0.000000,0.000000,0.000000,0.000000,0.000000,0.298454,0.274846,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.281436,0.000000,0.343006,0.000000,0.000000,0.000000,0.287786,0.000000,0.326264,0.000000
8,0.000000,0.000000,0.290411,0.000000,0.000000,0.000000,0.355164,0.000000,0.320000,0.000000,0.000000,0.349650
9,0.359531,0.000000,0.000000,0.000000,0.290444,0.000000,0.314663,0.000000,0.348626,0.000000,0.000000,0.000000
10,0.000000,0.345471,0.292517,0.000000,0.000000,0.000000,0.301169,0.000000,0.395319,0.000000,0.000000,0.000000


In [277]:
Promo_Sensitivity


array([0.97671939, 0.76227342, 0.81819194, 0.83515284, 0.8657577 ,
       0.89283139, 0.95198538, 0.80268148, 0.90668868, 0.87407859])